In [2]:
# imports etc.

import sys, os
sys.path.append(os.path.abspath('../'))

from src.utils.flow import *
from src.utils.flow_calculations import *
from src.utils.restoration import *
from src.utils.truncated_packet import *
from src.operations.size_perturbation_logic import *
from src.operations.timing_perturbation_logic import *
from src.operations.calculate_fitness import *

In [3]:
# data preparation

In [4]:
captures = [
    "botnet-capture-20110811-neris",
    "botnet-capture-20110812-rbot",
    "botnet-capture-20110815-fast-flux-2",
    "botnet-capture-20110815-fast-flux",
    "botnet-capture-20110815-rbot-dos",
    "botnet-capture-20110816-donbot",
    "botnet-capture-20110816-qvod",
    "botnet-capture-20110816-sogou",
    "botnet-capture-20110817-bot",
    "botnet-capture-20110818-bot-2",
    "botnet-capture-20110819-bot"
]

In [6]:
# for each capture mean flow which will be tested on the corresponding model

import numpy as np

def mean_dicts(dicts):
    # Get all keys from all dictionaries
    keys = set(k for d in dicts for k in d)
    # Initialize a new dictionary to store mean values
    mean_dict = {}
    for key in keys:
        # Calculate the mean for each key
        mean_dict[key] = np.mean([d[key] for d in dicts if key in d])
    return mean_dict

mean_flows = []

for i in range(len(captures)):
    print(f'Working on capture: {captures[i]}')

    pcap_file_path = f'../data/raw/filtered/{captures[i]}.pcap'
    model_name = f'{captures[i]}'

    # Load truncated packets from pcap file
    truncated_packets = create_truncated_packets_from_pcap(pcap_file_path)
    print(f"Total packets loaded: {len(truncated_packets)}")

    # Assign flow IDs to packets
    truncated_packets = assign_flow_ids_to_packets(truncated_packets)

    # Extract unique flow IDs
    flow_ids = set(packet.flow_id for packet in truncated_packets)
    flows = []

    for fid in flow_ids:
        # For each flow ID, prepare statistical values and append to the list
        sizing_stats = prepare_size_stats(truncated_packets, fid)
        timing_stats = prepare_timing_stats(truncated_packets, fid)
        flow_parameters = [sizing_stats, timing_stats]
        flows.append(flow_parameters)
    
    # Calculate mean values for sizing_stats and timing_stats
    mean_sizing_stats = mean_dicts([flow[0] for flow in flows])
    mean_timing_stats = mean_dicts([flow[1] for flow in flows])
    
    # Add mean values to mean_flows
    mean_flows.append([mean_sizing_stats, mean_timing_stats])

print("Mean flows calculated:")
print(mean_flows)


            

Working on capture: botnet-capture-20110811-neris
Total packets loaded: 4955
hash groups: 593
Working on capture: botnet-capture-20110812-rbot
Total packets loaded: 4958
hash groups: 449
Working on capture: botnet-capture-20110815-fast-flux-2
Total packets loaded: 4973
hash groups: 366
Working on capture: botnet-capture-20110815-fast-flux
Total packets loaded: 4968
hash groups: 139
Working on capture: botnet-capture-20110815-rbot-dos
Total packets loaded: 2653
hash groups: 85
Working on capture: botnet-capture-20110816-donbot
Total packets loaded: 4952
hash groups: 657
Working on capture: botnet-capture-20110816-qvod
Total packets loaded: 4943
hash groups: 57
Working on capture: botnet-capture-20110816-sogou
Total packets loaded: 4997
hash groups: 35
Working on capture: botnet-capture-20110817-bot
Total packets loaded: 4843
hash groups: 145
Working on capture: botnet-capture-20110818-bot-2
Total packets loaded: 1200
hash groups: 242
Working on capture: botnet-capture-20110819-bot
Total

In [ ]:
# IN ORDER TO PREDICT (FOR TARGET MODEL): predict_single_flow_target(model_name, flow_parameters[0], flow_parametrs[1])

In [ ]:
# data testing

In [ ]:
# target values

In [ ]:
# 

In [ ]:
# xai preparation & perturbation functions

# 

#adjust_packet_size(packets, flow_id, params['direction'], 'normal', params['scaling_factor'], params['focus_point'])
#adjust_packet_size(packets, flow_id, params['direction'], 'uniform', params['scaling_factor'], params['focus_point'])
#apply_time_perturbation_with_focus(packets, flow_id, 'normal', params['focus_point'], params['scaling_factor'])
#apply_time_perturbation_with_focus(packets, flow_id, 'uniform', params['focus_point'], params['scaling_factor'])


In [ ]:
# xai visualizations

In [ ]:
# final cost analysis visualization